In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import sys
import os

In [3]:
from models import *

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/paperspace/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [4]:
from fastai.column_data import *

In [5]:
from fastai.conv_learner import *
from fastai.dataset import *

In [6]:
import numpy as np

In [ ]:
ls -f KITTI/ | grep .csv

In [ ]:
ls KITTI/2011_10_03/2011_10_03_drive_0034_sync/image_03/data/0000003783.png

train, val= pd.read_csv('./KITTI/training.csv'), pd.read_csv('./KITTI/validation.csv')

In [7]:
train, val= pd.read_csv('./KITTI/training_192_640.csv'), pd.read_csv('./KITTI/validation_192_640.csv')

In [8]:
train.head()

,t0,t1,t2,fx,fy,cx,cy
0,2011_09_29/2011_09_29_drive_0071_sync/image_02...,2011_09_29/2011_09_29_drive_0071_sync/image_02...,2011_09_29/2011_09_29_drive_0071_sync/image_02...,370.454846,366.809413,309.628545,92.687081
1,2011_09_29/2011_09_29_drive_0071_sync/image_02...,2011_09_29/2011_09_29_drive_0071_sync/image_02...,2011_09_29/2011_09_29_drive_0071_sync/image_02...,370.454846,366.809413,309.628545,92.687081
2,2011_09_29/2011_09_29_drive_0071_sync/image_02...,2011_09_29/2011_09_29_drive_0071_sync/image_02...,2011_09_29/2011_09_29_drive_0071_sync/image_02...,370.454846,366.809413,309.628545,92.687081
3,2011_09_29/2011_09_29_drive_0071_sync/image_02...,2011_09_29/2011_09_29_drive_0071_sync/image_02...,2011_09_29/2011_09_29_drive_0071_sync/image_02...,370.454846,366.809413,309.628545,92.687081
4,2011_09_29/2011_09_29_drive_0071_sync/image_02...,2011_09_29/2011_09_29_drive_0071_sync/image_02...,2011_09_29/2011_09_29_drive_0071_sync/image_02...,370.454846,366.809413,309.628545,92.687081


In [9]:
trn0, trn1, trn2, trn_camera = train.t0, train.t1,train.t2, train[['fx', 'fy', 'cx', 'cy']]

In [10]:
val0, val1, val2, val_camera  = val.t0, val.t1, val.t2, val[['fx', 'fy', 'cx', 'cy']]

In [ ]:
trn_camera.head()

val_idxs = get_cv_idxs(len(fnames), val_pct=min(0.01/keep_pct, 0.1))
((val_x,trn_x),(val_y,trn_y)) = split_by_idx(val_idxs, np.array(fnames), np.array(fnames))
len(val_x),len(trn_x)

In [11]:
aug_tfms = [
    #RandomDihedral(tfm_y=TfmType.NO),
    RandomLighting(b=0.05, c=0.05)]


In [12]:
arch = f
scale,bs = 1, 4
# scale,bs = 4,32
sz = None #np.array([345, 1242]) 
sz_y = None 

PATH = 'Fastai_TRN'

In [13]:
class UnsupFilesDataset(FilesDataset):
    def __init__(self, fnames, y, transform, path):
        super().__init__(fnames, transform, path)
    def get_y(self, i): return None
    def get_c(self): return 0

In [14]:
class UnShuffleImageData(ImageData):
    def __init__(self, path, datasets, bs, num_workers, classes):
        trn_ds,val_ds,fix_ds,aug_ds,test_ds,test_aug_ds = datasets
        self.path,self.bs,self.num_workers,self.classes = path,bs,num_workers,classes
        self.trn_dl,self.val_dl,self.fix_dl,self.aug_dl,self.test_dl,self.test_aug_dl = [
            self.get_dl(ds,shuf) for ds,shuf in [
                (trn_ds,False),(val_ds,False),(fix_ds,False),(aug_ds,False),
                (test_ds,False),(test_aug_ds,False)
            ]
        ]

In [15]:
class UnScaleTransforms():
    def __init__(self, sz, tfms, normalizer, denorm,
                 tfm_y=TfmType.NO, sz_y=None):
        if sz_y is None: sz_y = sz
        self.sz,self.denorm,self.norm,self.sz_y = sz,denorm,normalizer,sz_y
        self.tfms = tfms
        if normalizer is not None: self.tfms.append(normalizer)
        self.tfms.append(ChannelOrder(tfm_y))

    def __call__(self, im, y=None): return compose(im, y, self.tfms)
    def __repr__(self): return str(self.tfms)

class Transforms():
    def __init__(self, sz, tfms, normalizer, denorm, crop_type=CropType.CENTER,
                 tfm_y=TfmType.NO, sz_y=None):
        if sz_y is None: sz_y = sz
        self.sz,self.denorm,self.norm,self.sz_y = sz,denorm,normalizer,sz_y
        crop_tfm = crop_fn_lu[crop_type](sz, tfm_y, sz_y)
        self.tfms = tfms
        self.tfms.append(crop_tfm)
        if normalizer is not None: self.tfms.append(normalizer)
        self.tfms.append(ChannelOrder(tfm_y))

    def __call__(self, im, y=None): return compose(im, y, self.tfms)
    def __repr__(self): return str(self.tfms)
    

def image_gen(normalizer, denorm, sz, tfms=None, max_zoom=None, pad=0, crop_type=None,
              tfm_y=None, sz_y=None, pad_mode=cv2.BORDER_REFLECT, scale=None):
    """
    Generate a standard set of transformations

    Arguments
    ---------
     normalizer :
         image normalizing function
     denorm :
         image denormalizing function
     sz :
         size, sz_y = sz if not specified.
     tfms :
         iterable collection of transformation functions
     max_zoom : float,
         maximum zoom
     pad : int,
         padding on top, left, right and bottom
     crop_type :
         crop type
     tfm_y :
         y axis specific transformations
     sz_y :
         y size, height
     pad_mode :
         cv2 padding style: repeat, reflect, etc.

    Returns
    -------
     type : ``Transforms``
         transformer for specified image operations.

    See Also
    --------
     Transforms: the transformer object returned by this function
    """
    if tfm_y is None: tfm_y=TfmType.NO
    if tfms is None: tfms=[]
    elif not isinstance(tfms, collections.Iterable): tfms=[tfms]
    if sz_y is None: sz_y = sz
    if scale is None:
        scale = [RandomScale(sz, max_zoom, tfm_y=tfm_y, sz_y=sz_y) if max_zoom is not None
                 else Scale(sz, tfm_y, sz_y=sz_y)]
    elif not is_listy(scale): scale = [scale]
    if pad: scale.append(AddPadding(pad, mode=pad_mode))
    if crop_type!=CropType.GOOGLENET: tfms=scale+tfms
    return Transforms(sz, tfms, normalizer, denorm, crop_type,
                      tfm_y=tfm_y, sz_y=sz_y)

def noop(x):
    """dummy function for do-nothing.
    equivalent to: lambda x: x"""
    return x

transforms_basic    = [RandomRotate(10), RandomLighting(0.05, 0.05)]
transforms_side_on  = transforms_basic + [RandomFlip()]
transforms_top_down = transforms_basic + [RandomDihedral()]

imagenet_stats = A([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
"""Statistics pertaining to image data from image net. mean and std of the images of each color channel"""
inception_stats = A([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
inception_models = (inception_4, inceptionresnet_2)

def tfms_from_stats(stats, sz, aug_tfms=None, max_zoom=None, pad=0, crop_type=CropType.RANDOM,
                    tfm_y=None, sz_y=None, pad_mode=cv2.BORDER_REFLECT, norm_y=True, scale=None):
    """ Given the statistics of the training image sets, returns separate training and validation transform functions
    """
    if aug_tfms is None: aug_tfms=[]
    tfm_norm = Normalize(*stats, tfm_y=tfm_y if norm_y else TfmType.NO) if stats is not None else None
    tfm_denorm = Denormalize(*stats) if stats is not None else None
    val_crop = CropType.CENTER if crop_type in (CropType.RANDOM,CropType.GOOGLENET) else crop_type
    val_tfm = image_gen(tfm_norm, tfm_denorm, sz, pad=pad, crop_type=val_crop,
            tfm_y=tfm_y, sz_y=sz_y, scale=scale)
    trn_tfm = image_gen(tfm_norm, tfm_denorm, sz, pad=pad, crop_type=crop_type,
            tfm_y=tfm_y, sz_y=sz_y, tfms=aug_tfms, max_zoom=max_zoom, pad_mode=pad_mode, scale=scale)
    return trn_tfm, val_tfm


def tfms_from_model(f_model, sz, aug_tfms=None, max_zoom=None, pad=0, crop_type=CropType.RANDOM,
                    tfm_y=None, sz_y=None, pad_mode=cv2.BORDER_REFLECT, norm_y=True, scale=None):
    """ Returns separate transformers of images for training and validation.
    Transformers are constructed according to the image statistics given by the model. (See tfms_from_stats)

    Arguments:
        f_model: model, pretrained or not pretrained
    """
    stats = inception_stats if f_model in inception_models else imagenet_stats
    return tfms_from_stats(stats, sz, aug_tfms, max_zoom=max_zoom, pad=pad, crop_type=crop_type,
                           tfm_y=tfm_y, sz_y=sz_y, pad_mode=pad_mode, norm_y=norm_y, scale=scale)


    return Transforms(sz, tfms, normalizer, denorm, crop_type,
                      tfm_y=tfm_y, sz_y=sz_y, )

In [16]:
stats = imagenet_stats
tfm_norm = Normalize(*stats, tfm_y=TfmType.NO) 
tfm_denorm = Denormalize(*stats)

In [17]:
trn_tfms = UnScaleTransforms(sz=None, tfms=[], normalizer=tfm_norm, denorm=tfm_denorm, tfm_y=TfmType.NO, sz_y=sz_y)
val_tfms = UnScaleTransforms(sz=None, tfms=[], normalizer=tfm_norm, denorm=tfm_denorm, tfm_y=TfmType.NO, sz_y=sz_y)

In [18]:
def get_MD(trn, val):
    
#     tfms = tfms_from_model(
#         arch,
#         sz,
#         crop_type=CropType.NO,
#         tfm_y=TfmType.NO,
#         aug_tfms=aug_tfms,
#         sz_y=sz_y)

    tfms = (trn_tfms, val_tfms)
    
    datasets = ImageData.get_ds(
        UnsupFilesDataset,
        (trn, None),
        (val, None),
        tfms, path='KITTI/')
    md = UnShuffleImageData(
        PATH,
        datasets,
        bs, num_workers=16,
        classes=None)
    
    return md

In [19]:
def get_cam(trn_cam, val_cam):
    return ColumnarModelData.from_data_frames(
        path= None,
        trn_df=trn_cam, val_df=val_cam,
        trn_y=None, val_y=None,
        cat_flds=[],
        bs=bs, shuffle=False)

In [20]:
MD1 = get_MD(trn0, val0)
MD2 = get_MD(trn1, val1)
MD3 = get_MD(trn2, val2)
MDcam = get_cam(trn_camera, val_camera)

In [21]:
class Loss(nn.Module):
    def __init__(self, scale=0.5, Tscale=1):
        super().__init__()
        self.appr = TriAppearanceLoss(scale=Tscale)
        self.smooth = SmoothLoss()
        self.scale = scale
    def forward(self, d1, d2 ,d3, poses_x2, x1, x2, x3, camera):
        appr_loss, details = self.appr(d1, d3, poses_x2, x1, x2, x3, camera)
        smooth_loss = (self.smooth(d1) +self.smooth(d2) + self.smooth(d3)) * self.scale
        #print(type(appr_loss))
        #print(type(smooth_loss))
        return appr_loss + smooth_loss, (appr_loss, smooth_loss, *details) 
               
        

In [35]:
m = TriDepth(get_base(), 1).cuda()

In [36]:
l = Loss().cuda() 

In [37]:
opt = optim.SGD(m.parameters(), lr=0.0001, momentum=0.9)
#opt = optim.Adam(m.parameters())

In [25]:
import matplotlib.pyplot as plt

In [ ]:
imgbatch1 = next(iter(MD1.trn_dl))
imgbatch2 = next(iter(MD2.trn_dl))
imgbatch3 = next(iter(MD3.trn_dl))
cambatch = next(iter(MDcam.trn_dl))

img1 = MD1.trn_ds.denorm(imgbatch1)[0]
img2 = MD2.trn_ds.denorm(imgbatch2)[0]
img3 = MD3.trn_ds.denorm(imgbatch3)[0]
cam = cambatch[1][0]

f, axs = plt.subplots(1, 3, figsize=(12, 6))
axs[0].imshow(img1)
axs[1].imshow(img2)
axs[2].imshow(img3)

print(cam.cpu().numpy().reshape(-1))

In [ ]:
DL1, DL2, DL3, DLcam = iter(MD1.trn_dl), iter(MD2.trn_dl), iter(MD3.trn_dl), iter(MDcam.trn_dl)
imgs1, imgs2, imgs3, cam = V([next(DL1), next(DL2),next(DL3), next(DLcam)[1]])

In [ ]:
d1, d2, d3, poses_x2 = m(imgs1, imgs2, imgs3)

In [ ]:
loss, details = l(d1, d2, d3, poses_x2, imgs1, imgs2, imgs3, cam)
loss.backward()
opt.step()
losses.append(loss)

In [26]:
verbose = 20
def fit(model, metric, opt, MD1, MD2, MD3, MDcam):
    DL1, DL2, DL3, DLcam = iter(MD1.trn_dl), iter(MD2.trn_dl), iter(MD3.trn_dl), iter(MDcam.trn_dl)
    losses = []
    for i in range(len(MD1.trn_ds)//bs):
        opt.zero_grad()
        imgs1, imgs2, imgs3, cam = V([next(DL1), next(DL2),next(DL3), next(DLcam)[1]])
        
        d1, d2, d3, poses_x2 = m(imgs1, imgs2, imgs3)
        loss, details = l(d1, d2, d3, poses_x2, imgs1, imgs2, imgs3, cam)
        loss.backward()
        opt.step()
        losses.append(loss.data[0])
        if i%verbose == 0: print(loss.data[0],
                                 details[0].data[0],
                                 details[1].data[0],
                                 details[2].data[0],
                                 details[3].data[0],
                                 sep='\t')
    return A(loesses).mean()

In [38]:
fit(m, l, opt, MD1, MD2, MD3, MDcam)

3.125248670578003	3.0441842079162598	0.08106439560651779	0.004386676475405693	3.03979754447937
3.0934057235717773	3.0114824771881104	0.08192313462495804	0.012196463532745838	2.999285936355591
2.708221435546875	2.6284539699554443	0.07976752519607544	0.009869917295873165	2.618584156036377
2.936650037765503	2.858968734741211	0.07768139243125916	0.011930396780371666	2.8470382690429688
2.8412442207336426	2.7657363414764404	0.07550782710313797	0.008165089413523674	2.757571220397949
2.8085575103759766	2.7332379817962646	0.07531944662332535	0.010053958743810654	2.723184108734131
2.5861833095550537	2.5119495391845703	0.0742337629199028	0.013265728019177914	2.4986839294433594
2.5859150886535645	2.5129265785217285	0.07298856973648071	0.012116143479943275	2.500810384750366
2.2553679943084717	2.1849300861358643	0.07043779641389847	0.012508852407336235	2.1724212169647217
2.2906947135925293	2.222074270248413	0.06862039864063263	0.017610225826501846	2.2044639587402344
2.197402000427246	2.1319327354431

2.4516355991363525	2.428452730178833	0.02318275347352028	0.0005891022738069296	2.427863597869873
2.920832633972168	2.898055076599121	0.02277759648859501	0.0012491375673562288	2.896806001663208
2.729867696762085	2.7065980434417725	0.023269692435860634	0.001067107543349266	2.705530881881714
2.4087438583374023	2.3859848976135254	0.022759078070521355	0.0014464345294982195	2.384538412094116
2.6462554931640625	2.625364065170288	0.020891522988677025	0.0009505139314569533	2.62441349029541
2.9820847511291504	2.9616246223449707	0.020460106432437897	0.00156216521281749	2.9600625038146973
2.9476988315582275	2.926788091659546	0.02091069333255291	0.000726594589650631	2.9260613918304443
2.9128687381744385	2.8920249938964844	0.020843682810664177	0.0007984819239936769	2.8912265300750732
2.3440239429473877	2.3223423957824707	0.021681614220142365	0.0014632309321314096	2.3208792209625244
2.4037387371063232	2.383716106414795	0.020022524520754814	0.0004898144979961216	2.3832263946533203
2.728020668029785	2.

2.780453681945801	2.7701356410980225	0.010318136774003506	0.0017461872193962336	2.7683894634246826
2.4235098361968994	2.412217378616333	0.011292455717921257	0.0004251972713973373	2.411792278289795
2.3864362239837646	2.374846935272217	0.011589208617806435	0.0010118527570739388	2.3738350868225098
2.733649730682373	2.7219748497009277	0.01167491264641285	0.00019634711497928947	2.721778392791748
2.7478060722351074	2.7353148460388184	0.012491291388869286	0.00019262367277406156	2.7351222038269043
2.386340856552124	2.371182680130005	0.01515816431492567	0.0002823908580467105	2.3709003925323486
2.5779101848602295	2.565169095993042	0.012741165235638618	0.00035930375452153385	2.564809799194336
3.237008571624756	3.226677656173706	0.010330884717404842	4.672315117204562e-05	3.226630926132202
3.249887466430664	3.2391505241394043	0.010736915282905102	2.6188810807070695e-05	3.239124298095703
2.7243435382843018	2.712601661682129	0.011741932481527328	0.0001796556607587263	2.7124218940734863
2.841555356979

2.7681989669799805	2.7605459690093994	0.007653066888451576	0.0002837697393260896	2.7602622509002686
2.498276472091675	2.4900588989257812	0.008217631839215755	0.0007544352556578815	2.489304542541504
2.4513204097747803	2.442854404449463	0.008466080762445927	0.00017316399316769093	2.442681312561035
2.6446468830108643	2.6371514797210693	0.007495451252907515	0.0011900801910087466	2.6359612941741943
2.6254591941833496	2.617548704147339	0.007910533808171749	0.0002120355929946527	2.6173367500305176
2.905384063720703	2.8985867500305176	0.00679740821942687	0.00025739186094142497	2.898329257965088
2.709402561187744	2.702704668045044	0.006697837729007006	0.0003104767529293895	2.7023942470550537
2.762194871902466	2.7548534870147705	0.00734136626124382	0.00030265888199210167	2.7545509338378906
2.217925548553467	2.2104380130767822	0.007487437687814236	0.0008552784565836191	2.209582805633545
2.2837014198303223	2.2761523723602295	0.0075490400195121765	0.0009980483446270227	2.2751543521881104
2.73096728

2.4374775886535645	2.4305925369262695	0.006885056383907795	0.00025162255042232573	2.4303410053253174
2.8628320693969727	2.8580307960510254	0.004801166709512472	0.0005027685547247529	2.8575279712677
2.5631093978881836	2.555758476257324	0.00735084805637598	6.129433313617483e-05	2.555697202682495
2.7660136222839355	2.7601706981658936	0.005842920392751694	9.925691119860858e-05	2.7600715160369873
2.4977738857269287	2.4908087253570557	0.0069650923833251	0.00011425127013353631	2.490694522857666
2.703688859939575	2.6978323459625244	0.005856587551534176	0.00014151554205454886	2.697690725326538
3.140843152999878	3.1362569332122803	0.004586158785969019	3.0506034818245098e-05	3.1362264156341553
2.0338151454925537	2.0263350009918213	0.007480216212570667	0.0009683219250291586	2.02536678314209
2.22105073928833	2.214275360107422	0.006775345653295517	0.0006500098388642073	2.213625431060791
2.8471152782440186	2.8425662517547607	0.004548960365355015	0.00024711742298677564	2.8423192501068115
2.52310204505

2.321258068084717	2.3162336349487305	0.005024522077292204	0.0009377434034831822	2.315295934677124
2.4743378162384033	2.468749761581421	0.005588095635175705	0.0002791930746752769	2.468470573425293
2.3572235107421875	2.3501222133636475	0.007101413793861866	4.654526856029406e-05	2.3500757217407227
2.322391986846924	2.3171751499176025	0.005216833204030991	0.00033162295585498214	2.3168435096740723
2.2385988235473633	2.2308614253997803	0.007737321313470602	3.8751728425268084e-05	2.2308225631713867
2.4118144512176514	2.405796527862549	0.006017928943037987	2.9388093025772832e-05	2.4057672023773193
2.743203639984131	2.7387869358062744	0.004416781943291426	0.00021248520351946354	2.738574504852295
2.9644272327423096	2.9601967334747314	0.004230409394949675	5.9323530877009034e-05	2.960137367248535
2.864637613296509	2.8599512577056885	0.004686254542320967	3.7999823689460754e-05	2.8599133491516113
2.1372487545013428	2.131835460662842	0.0054133376106619835	0.00033450592309236526	2.1315009593963623
2.7

2.8102595806121826	2.80495285987854	0.005306709557771683	2.663580744410865e-05	2.8049261569976807
2.9672653675079346	2.963195562362671	0.00406974833458662	3.577238385332748e-05	2.9631597995758057
2.151494026184082	2.1465041637420654	0.004989779554307461	0.0006841186550445855	2.145820140838623
2.539578914642334	2.534179925918579	0.005399065092206001	5.129586497787386e-05	2.5341286659240723
2.579021692276001	2.5736608505249023	0.005360879935324192	2.3952276023919694e-05	2.573637008666992
2.5620803833007812	2.556755542755127	0.005324891768395901	4.055887620779686e-05	2.5567150115966797
2.717970848083496	2.713256597518921	0.004714250564575195	0.00010116457269759849	2.713155508041382
3.130608320236206	3.1259663105010986	0.004641897976398468	2.7580887035583146e-05	3.125938653945923
2.1231067180633545	2.118786334991455	0.004320305306464434	0.0007329236250370741	2.118053436279297
2.4198710918426514	2.4151217937469482	0.004749230109155178	6.560729525517672e-05	2.4150562286376953
2.5153150558471

2.2343432903289795	2.2296926975250244	0.004650546703487635	0.0005536326789297163	2.2291390895843506
2.6443793773651123	2.639859199523926	0.00452017318457365	6.369344919221476e-05	2.6397955417633057
2.601379871368408	2.596928119659424	0.004451774060726166	7.848192035453394e-05	2.5968496799468994
2.8553178310394287	2.851792097091675	0.003525753505527973	8.180950680980459e-05	2.851710319519043
2.5569229125976562	2.553309440612793	0.0036134368274360895	0.00016936052998062223	2.553140163421631
2.203590154647827	2.200425148010254	0.0031650755554437637	0.0006998782628215849	2.1997251510620117
2.245840072631836	2.242053747177124	0.0037863480392843485	0.0027428087778389454	2.2393109798431396
2.387314558029175	2.38415789604187	0.0031566335819661617	0.0023205631878226995	2.3818373680114746
2.6027321815490723	2.598778247833252	0.003954033367335796	2.8502936402219348e-05	2.5987496376037598
2.3175697326660156	2.3137879371643066	0.003781769657507539	0.0002333338197786361	2.313554525375366
2.464531898

2.2104809284210205	2.2075631618499756	0.002917653415352106	0.0014078875537961721	2.206155300140381
2.7399191856384277	2.7373270988464355	0.002591998316347599	0.0006925273919478059	2.7366344928741455
3.095137119293213	3.0930018424987793	0.0021352614276111126	6.298996595432982e-05	3.0929388999938965
2.6877245903015137	2.6850807666778564	0.00264389393851161	0.00043972578714601696	2.684641122817993
2.535869598388672	2.5331711769104004	0.0026984205469489098	0.0010870852274820209	2.5320839881896973
2.728853464126587	2.7264564037323	0.0023971283808350563	0.00048096265527419746	2.725975513458252
2.8948616981506348	2.8925867080688477	0.002274960745126009	0.0001284119498450309	2.892458200454712
2.8655591011047363	2.86344313621521	0.0021158906165510416	5.2987867093179375e-05	2.8633902072906494
2.805729389190674	2.802928924560547	0.0028004790656268597	0.00011345042730681598	2.8028154373168945
2.3897745609283447	2.385561227798462	0.004213426262140274	0.000332200521370396	2.3852291107177734
2.341434

KeyboardInterrupt: 

In [ ]:
DL1, DL2, DL3, DLcam = iter(MD1.val_dl), iter(MD2.val_dl), iter(MD3.val_dl), iter(MDcam.val_dl)
imgs1, imgs2, imgs3, cam = V([next(DL1), next(DL2),next(DL3), next(DLcam)[1]])

In [ ]:
d1, d2, d3, poses_x2 = m(imgs1, imgs2, imgs3)

In [ ]:
def plot_depth(depth):
    depth = depth.cpu().data[0].permute(1,2,0).numpy()
    plt.imshow(depth)
    plt.axis('off')

In [ ]:
d1.shape

In [ ]:
depth = d1.cpu().data[0].permute(1,2,0).numpy()

In [ ]:
depth[:,:,0].shape

In [ ]:
plt.imshow(depth[:,:,0])

In [ ]:
depth.mean()

In [ ]:
plot_depth(d1)